<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Import Libraries&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [2]:
!pip install keras --quiet

In [3]:
# Core libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, BatchNormalization,
    Embedding, LSTM, GRU, SimpleRNN,
    GlobalMaxPooling1D, Conv1D, MaxPooling1D,
    Flatten, Bidirectional, SpatialDropout1D
)
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn import decomposition, metrics, pipeline


2025-06-20 09:48:45.677308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750412925.891177      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750412925.955092      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Read the data&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [ ]:
train = pd.read_csv('/path/to/tarin.csv')
train = pd.read_csv('/path/to/test.csv')

In [5]:
import ast
train['Categories'] = train['Categories'].apply(lambda x: ast.literal_eval(x))
train.head()

,Id,Title,Abstract,Categories
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO]
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR]
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]"
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]"
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]"


In [6]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(train['Categories'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
multilabel.classes_

array(['cs.AI', 'cs.AR', 'cs.CE', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.DB',
       'cs.DC', 'cs.DM', 'cs.GT', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO',
       'cs.NI', 'cs.OS', 'cs.PL', 'cs.RO', 'cs.SD', 'cs.SE', 'econ.EM',
       'econ.GN', 'econ.TH', 'eess.AS', 'eess.IV', 'eess.SP', 'math.AC',
       'math.AP', 'math.AT', 'math.CO', 'math.CV', 'math.GR', 'math.IT',
       'math.LO', 'math.NT', 'math.PR', 'math.QA', 'math.ST', 'q-bio.BM',
       'q-bio.CB', 'q-bio.GN', 'q-bio.MN', 'q-bio.NC', 'q-bio.TO',
       'q-fin.CP', 'q-fin.EC', 'q-fin.GN', 'q-fin.MF', 'q-fin.PM',
       'q-fin.PR', 'q-fin.RM', 'q-fin.TR', 'stat.AP', 'stat.CO',
       'stat.ME', 'stat.ML', 'stat.TH'], dtype=object)

In [8]:
y = pd.DataFrame(y, columns=multilabel.classes_)
y

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51206,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51208,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
train['comment_text'] = train['Title'] + '. ' + train['Abstract']
train = train['comment_text']
test['comment_text'] = test['Title'] + '. ' + test['Abstract']
#test = test['comment_text']

In [10]:
train = pd.concat([train, y], axis=1, join='inner')

In [ ]:
print(train.shape)

(51210, 58)
(10974, 4)


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Do some cleaning on the data&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [13]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

In [14]:
train['comment_text']=train['comment_text'].apply(lambda x:strip_links(x))
test['comment_text']=test['comment_text'].apply(lambda x:strip_links(x))

In [15]:
### replace :\n 
train['comment_text']=train['comment_text'].str.replace("\n",' ')

In [16]:
### replace :\n 
test['comment_text']=test['comment_text'].str.replace("\n",' ')

In [17]:
# Define the function to remove the punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
# Apply to the DF series
train['comment_text'] = train['comment_text'].apply(remove_punctuations) 

In [18]:
# Apply to the DF series
test['comment_text'] = test['comment_text'].apply(remove_punctuations) 

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Train and Val split&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [19]:
y

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51206,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51208,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(train.comment_text.values, y,  
                                                  random_state=42, 
                                                  test_size=0.2)

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Define Vocab size and input string size&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [21]:
## Check lenght of text in the data
train['comment_text'].apply(lambda x:len(str(x).split())).max()

452

In [22]:
max_features = 5000
maxlen = 500

In [23]:
token=tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
token.fit_on_texts(train.comment_text)

In [24]:
X_train_seq=token.texts_to_sequences(X_train)
X_test_seq=token.texts_to_sequences(X_test)

In [25]:
#zero pad the sequences
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)

In [26]:
word_index = token.word_index

In [27]:
len(token.word_index)##251102

154103

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Word2Vec embeddings&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [28]:
# from gensim.models import Word2Vec, KeyedVectors
# # Load pretrained Glove model (in word2vec form)
# word2vec_model = KeyedVectors.load_word2vec_format("/kaggle/input/embedings/GoogleNews-vectors-negative300.bin", binary=True)

In [29]:
from gensim.models import Word2Vec, KeyedVectors
# Load pretrained Glove model (in word2vec form)
word2vec_model = KeyedVectors.load_word2vec_format("/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin", binary=True)

In [30]:
#Embedding length based on selected model - we are using 50d here.
embedding_vector_length = 300

In [31]:
#Initialize embedding matrix
embedding_matrix = np.zeros((max_features + 1, embedding_vector_length))
print(embedding_matrix.shape)

(5001, 300)


In [32]:
for word, i in sorted(token.word_index.items(),key=lambda x:x[1]):
    if i > (max_features+1):
        break
    try:
        embedding_vector = word2vec_model[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [33]:
max_features

5000

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Model Building Using Word2vec&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [34]:
# A simpleRNN without any pretrained embeddings and one dense layer
model = Sequential()
model.add(Embedding(max_features + 1, #Vocablury size
                    embedding_vector_length, #Embedding size
                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                    input_length=maxlen))  # Number of words in each review

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))

# Another LSTM layer
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))

# Dense layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer for 57-way multilabel classification
model.add(Dense(57, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',Precision(), Recall()])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1750413042.272219      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750413042.272878      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       1,500,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,500,300 (5.72 MB)

 Trainable params: 1,500,300 (5.72 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
history = model.fit(X_train_pad,
                    y_train,
                    epochs=20,
                    batch_size=32,          
                    validation_data=(X_test_pad, y_test))

Epoch 1/20


I0000 00:00:1750413050.819841     108 cuda_dnn.cc:529] Loaded cuDNN version 90300


1281/1281 ━━━━━━━━━━━━━━━━━━━━ 121s 87ms/step - accuracy: 0.0830 - loss: 0.1697 - precision: 0.0998 - recall: 0.0309 - val_accuracy: 0.1891 - val_loss: 0.1043 - val_precision: 0.6079 - val_recall: 0.0251
Epoch 2/20
1281/1281 ━━━━━━━━━━━━━━━━━━━━ 113s 89ms/step - accuracy: 0.1888 - loss: 0.1064 - precision: 0.5298 - recall: 0.0703 - val_accuracy: 0.3023 - val_loss: 0.0860 - val_precision: 0.6993 - val_recall: 0.2670
Epoch 3/20
1281/1281 ━━━━━━━━━━━━━━━━━━━━ 114s 89ms/step - accuracy: 0.3089 - loss: 0.0874 - precision: 0.7235 - recall: 0.2798 - val_accuracy: 0.3562 - val_loss: 0.0753 - val_precision: 0.6945 - val_recall: 0.3876
Epoch 4/20
1281/1281 ━━━━━━━━━━━━━━━━━━━━ 114s 89ms/step - accuracy: 0.3525 - loss: 0.0769 - precision: 0.7422 - recall: 0.3519 - val_accuracy: 0.3828 - val_loss: 0.0694 - val_precision: 0.7670 - val_recall: 0.4063
Epoch 5/20
1281/1281 ━━━━━━━━━━━━━━━━━━━━ 114s 89ms/step - accuracy: 0.3882 - loss: 0.0709 - precision: 0.7506 - recall: 0.4005 - val_accuracy: 0.4089 

In [36]:
from tensorflow.keras.models import Sequential

# Assume model is your Keras model
model.save('model.h5')

In [37]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_input(text, tokenizer, maxlen):
    text = strip_links(text)
    text = text.replace("\n", ' ')
    text = remove_punctuations(text)
    seq = tokenizer.texts_to_sequences([text])
    pad_seq = pad_sequences(seq, maxlen=maxlen)
    return pad_seq

def predict_categories(model, text, tokenizer, maxlen, multilabel_binarizer):
    processed_text = preprocess_input(text, tokenizer, maxlen)
    prediction = model.predict(processed_text)
    thresholded_prediction = prediction > 0.5

    # Check if any label has a value greater than 0.5
    if thresholded_prediction.any():
        predicted_labels = multilabel_binarizer.inverse_transform(thresholded_prediction)
    else:
        # Get the label with the highest value if no label is above the threshold
        max_label_index = prediction.argmax()
        max_label = [multilabel_binarizer.classes_[max_label_index]]
        predicted_labels = [max_label]

    return predicted_labels

# Example usage
input_string1 = "Two improvements in Birch's theorem on forms"
input_string2 = "Let K be a Birch field, that is, a field for which every diagonal form of odd degree in sufficiently many variables admits a non-zero solution; for example, K could be the field of rational numbers. Let f1,…,fr be homogeneous forms of odd degree over K in n variables, and let Z be the variety they cut out. Birch proved if n is sufficiently large then Z(K) contains a non-zero point. We prove two results which show that Z(K) is actually quite large. First, the Zariski closure of Z(K) has bounded codimension in An. And second, if the fi's have sufficiently high strength then Z(K) is in fact Zariski dense in Z. The proofs use recent results on strength, and our methods build on recent work of Bik, Draisma, and Snowden, which established similar improvements to Brauer's theorem on forms."
custom_input =  input_string1 + '. ' + input_string2

# Example usage for the second model
predicted_categories2 = predict_categories(model, custom_input, token, maxlen, multilabel)
print("Predicted categories for custom input using model2:", predicted_categories2)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
Predicted categories for custom input using model2: [('math.NT',)]
